## Deep Convolutional Generative Adverserial Networks

#### Imports

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.transforms import v2 as Transforms
from torch.utils.tensorboard import SummaryWriter

#### Architecture

In [23]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1), # stride 2 and padding 1 halves the dim
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d*2, kernel_size=4, stride=2, padding=1),
            self._block(features_d*2, features_d*4, kernel_size=4, stride=2, padding=1),
            self._block(features_d*4, features_d*8, kernel_size=4, stride=2, padding=1),
            nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid()
        )
    
    def _block(Self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            # Leaky ReLU in the Discriminator helps prevent dead neurons
            # and supports effective learning for distinguishing real
            # and fake images by allowing gradients for negative inputs.
    )
    
    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    # z_dim is the dimension of the noise vector
    def __init__(self, z_dim, channels_img, features_g):
        super().__init__()
        self.gen = nn.Sequential(
            self._block(z_dim, features_g*16, kernel_size=4, stride=1, padding=0), # 4 x 4
            self._block(features_g*16, features_g*8, kernel_size=4, stride=2, padding=1), # 8 x 8
            self._block(features_g*8, features_g*4, kernel_size=4, stride=2, padding=1), # 16 x 16
            self._block(features_g*4, features_g*2, kernel_size=4, stride=2, padding=1), # 32 x 32
            nn.ConvTranspose2d(features_g*2, channels_img, kernel_size=4, stride=2, padding=1),
            nn.Tanh() # works well with generator gradient optimization
        )

    def _block(Self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            # ReLU in the Generator promotes efficient training
            # and better output structure, allowing for a wider
            # range of activations necessary for generating complex images.
    )
    
    def forward(self, x):
        return self.gen(x)

#### Initialize weights for CNN

In [24]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, mean=0.0, std=0.02)

#### Hyperparameters

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 2e-4
Z_DIM = 100
IMAGE_SIZE = 64
CHANNELS_IMG = 1
NUM_EPOCHS = 5
BATCH_SIZE = 128
FEATURES_DISC = 64
FEATURES_GEN = 64

In [26]:
transforms = Transforms.Compose(
    [
        Transforms.Resize(IMAGE_SIZE),
        Transforms.ToTensor(),
        Transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]),
    ]
)
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [27]:
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn((32, Z_DIM, 1, 1)).to(device)

gen.train()
disc.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
    (6): Sigmoid()
  )
)

initialize architecture

In [ ]:
import datetime

log_dir_fake = f"logs/fake_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
log_dir_real = f"logs/real_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

writer_fake = SummaryWriter(log_dir_fake)
writer_real = SummaryWriter(log_dir_real)
step = 0

for epoch in range(NUM_EPOCHS):
    for batch_idx, (real, _) in enumerate(loader):
        # original shape of real = (32, 1, 28, 28)
        real = real.to(device) # flattening the 32x1x28x28 to 32x784
        noise = torch.randn((BATCH_SIZE, Z_DIM, 1, 1)).to(device)
        fake = gen(noise)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z))) ========================================

        # log(D(Real)) - coding this part
        disc_real = disc(real).view(-1)
        loss_dic_real = criterion(disc_real, torch.ones_like(disc_real)) # zeros because it is supposed to be real, so class 1

        # log(1 - D(G(z))) - coding this part
        disc_fake = disc(fake).view(-1)
        loss_dic_fake = criterion(disc_fake, torch.zeros_like(disc_fake)) # zeros because it is supposed to be fake, so class 0

        # finally
        lossD = (loss_dic_real + loss_dic_fake) / 2

        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) or better: max log(D(G(z))) ========================================
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        # tensorboard code
        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)

                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)
                img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                

                step += 1

Epoch [0/5] Batch 0/469                       Loss D: 0.2642, loss G: 3.9661
Epoch [1/5] Batch 0/469                       Loss D: 0.2206, loss G: 3.8722
Epoch [2/5] Batch 0/469                       Loss D: 0.2124, loss G: 2.9790
